In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import os

from datetime import date
print(date.today())

2025-12-01


In [14]:
workdir = '/mnt/disk7t/xwj/axolotl_rev'
# 准备结果目录和流程所需config文件
# level 1
output_path = f'{workdir}/result/dataset_gtex_nmd'
# level 2
samples_path = f'{output_path}/samples'
task_config_path = f'{output_path}/task_config'
task_output_path = f'{output_path}/task_output'
metric_output_path = f'{output_path}/metric'


tissues = pd.read_csv(f'{workdir}/gtex_processed/tissues.tsv', sep='\t', index_col=0)
tissues = tissues[ tissues['N_SAMPLE'] < 150 ]
tissues

,TISSUE_ID,TISSUE_NAME,N_SAMPLE,N_GENE,CTS_FILE,OUTLIER_FILE
8,BRNAMY,Brain - Amygdala,146,14492,/mnt/disk7t/xwj/axolotl_rev/gtex_processed/cts...,/mnt/disk7t/xwj/axolotl_rev/gtex_processed/out...
18,BRNSNG,Brain - Substantia nigra,132,14884,/mnt/disk7t/xwj/axolotl_rev/gtex_processed/cts...,/mnt/disk7t/xwj/axolotl_rev/gtex_processed/out...
28,KDNCTX,Kidney - Cortex,73,15406,/mnt/disk7t/xwj/axolotl_rev/gtex_processed/cts...,/mnt/disk7t/xwj/axolotl_rev/gtex_processed/out...
46,UTERUS,Uterus,139,14829,/mnt/disk7t/xwj/axolotl_rev/gtex_processed/cts...,/mnt/disk7t/xwj/axolotl_rev/gtex_processed/out...


In [11]:
from sklearn.metrics import auc
import pandas as pd
import numpy as np

## precision recall curve add segments for easy plotting
n_seg = 100
recall_seg = np.linspace(0, 1, num=n_seg+1)

def sort_only_score_auprc(res_data, score_col, df_outlier): 
    # for ABEILLE只对score排序，回答审稿人对ABEILLE bad质疑
    """
    计算ABEILLE方法单个打分方式的AUPRC
    参数:
        res_data: 包含预测结果的DataFrame
        score_col: 用于排序的打分列名
        df_outlier: 包含真实异常的DataFrame
    返回:
        auprc: AUPRC值
        pr_curve: 用于绘制的PR曲线点
    """
    # 按predict和score排序
    ascend = score_order[score_col]
    sorted_data = res_data.sort_values(by=[  score_col], ascending=[ascend])

    # 创建预测列
    sorted_data['rank'] = range(1, sorted_data.shape[0]+1)

    # 准备PR曲线数据
    pr_curve = sorted_data[['rank']].copy()
    # 标记真实的异常点
    pr_curve['label'] = 0
    pr_curve.loc[ zip(df_outlier['Sample'], df_outlier['Gene']), 'label'] = 1

    # 计算precision和recall
    pr_curve['precision'] = (pr_curve['label'].cumsum() / pr_curve['rank']).round(4)
    pr_curve['recall'] = (pr_curve['label'].cumsum() / pr_curve['label'].sum()).round(4)

    # 计算AUPRC
    auprc = auc(pr_curve['recall'], pr_curve['precision'])

    # 准备绘图用的曲线点
    pr_curve = pr_curve.drop_duplicates(subset=['precision', 'recall'])
    idx = [(pr_curve['recall'] - seg).abs().idxmin() for seg in recall_seg]
    
    return auprc, pr_curve.loc[idx]

def sort_label_score_auprc(res_data, class_col, score_col, df_outlier):
    """
    计算ABEILLE方法单个打分方式的AUPRC
    参数:
        res_data: 包含预测结果的DataFrame
        score_col: 用于排序的打分列名
        df_outlier: 包含真实异常的DataFrame
    返回:
        auprc: AUPRC值
        pr_curve: 用于绘制的PR曲线点
    """
    # 按predict和score排序
    ascend = score_order[score_col]
    sorted_data = res_data.sort_values(by=[ class_col, score_col], ascending=[False, ascend])

    # 创建预测列
    sorted_data['rank'] = range(1, sorted_data.shape[0]+1)

    # 准备PR曲线数据
    pr_curve = sorted_data[['rank',class_col, score_col]].copy()
    # 标记真实的异常点
    pr_curve['label'] = 0
    pr_curve.loc[ zip(df_outlier['Sample'], df_outlier['Gene']), 'label'] = 1

    # 计算precision和recall
    pr_curve['precision'] = (pr_curve['label'].cumsum() / pr_curve['rank']).round(4)
    pr_curve['recall'] = (pr_curve['label'].cumsum() / pr_curve['label'].sum()).round(4)

    # 计算AUPRC
    auprc = auc(pr_curve['recall'], pr_curve['precision'])

    # 准备绘图用的曲线点
    pr_curve = pr_curve.drop_duplicates(subset=['precision', 'recall'])
    idx = [(pr_curve['recall'] - seg).abs().idxmin() for seg in recall_seg]
    
    return auprc, pr_curve.loc[idx]


## 定义函数来计算AUPRC
def pred_outlier_auprc(res_data, df_outlier):
    
    res_data.index.name = 'Gene'
    res_data.columns.name = 'Sample'
    res_data = res_data.T.melt(ignore_index=False,value_name='pred').set_index(['Gene'],append=True).reorder_levels([1, 0])
    sorted_data = res_data.sort_values(by='pred',ascending=True)
    sorted_data['rank'] = range(1, sorted_data.shape[0]+1)
    # 准备PR曲线数据
    pr_curve = sorted_data[['rank','pred']].copy() # 保存pred值用于绘图20251201回答审稿人ABEILLE bad质疑
    pr_curve['label'] = 0
    
    for i, row in df_outlier.iterrows(): 
    #     print(i,row['Gene'],row['Sample'])
        pr_curve.loc[(row['Gene'], row['Sample']), 'label' ] = 1
    pr_curve['precision'] = (pr_curve['label'].cumsum()/pr_curve['rank']).round(4)
    pr_curve['recall'] = (pr_curve['label'].cumsum()/pr_curve['label'].sum()).round(4)

    # auprc value
    auprc = auc(pr_curve.loc[:,'recall'],pr_curve.loc[:,'precision'])
    
    # save a simple curve for plotting
    pr_curve = pr_curve.drop_duplicates(subset=['precision', 'recall'])
    # keep equally distributed points 
    idx = [ (pr_curve['recall'] - seg).abs().idxmin() for seg in recall_seg ]
    
    return auprc, pr_curve.loc[idx,]


# 定义方法和打分方式的对应关系
methods_scores = {
    'ABEILLE': ['predict', 'anomaly_score', 'delta_count', 'divergence_score'],
    'OUTRIDER': ['aberrant', 'pValue', 'zScore','padjust']
}
abl_scores = ['anomaly_score', 'delta_count', 'divergence_score']
outrider_scores = [ 'pValue', 'zScore','padjust']


# 定义排序逻辑
score_order = {
    'anomaly_score': False,
    'delta_count': True,
    'divergence_score': True,
    'pValue': True,
    'padjust': True,
    'zScore': True,
}


print(outrider_scores, abl_scores)


['pValue', 'zScore', 'padjust'] ['anomaly_score', 'delta_count', 'divergence_score']


In [18]:
# 为兼容以后的子集抽样，需要定义保存不同抽样方案的outlier
dict_outlier_setting = dict()
dict_outlier_setting['all'] = {}
simu = 0
for i, row in tissues.iterrows():
    # print(row)
    t  = row.TISSUE_ID
    tn = row.TISSUE_NAME
    ns = row.N_SAMPLE
    ng = row.N_GENE
    ctsfile = row.CTS_FILE
    outlierfile = row.OUTLIER_FILE

    dict_res = dict() # 单个组织的结果
    dict_auprc = dict()
    dict_curve = dict()
    
    for k in dict_outlier_setting.keys():
        # --> level 2. (tissue, task) one tissue have many tasks
        if k == 'all':
            prefix = f't{i:02d}_{t}_s{ns}_g{ng}' # id, number of samples, number of genes.
        else:
            pass # todo, add some change to prefix
        
        dict_res[k] = dict()
        dict_auprc[k] = dict()
        dict_curve[k] = dict()
        # 读入任务
        task_config = pd.read_csv(f'{task_config_path}/{prefix}.config', sep='\t')
        outliers_used = pd.read_csv(f'{outlierfile}', sep='\t', index_col=0)
        print(prefix, outliers_used.shape)
        
        baselines = [ 'ABEILLE', 'OUTRIDER', 'OUTSINGLE',]
        
        for m in baselines:
            # ==> level 3. (tissue, task, method)
            print(m)
            dict_res[k][m]= dict()
            for j,row in task_config.iterrows():
                # ==> level 4. (tissue, task, method)
                if 'Sample' in outliers_used.columns:
                    df_outlier = outliers_used
                else:
                    outlier_sample_col = 'SAMPLE_ID'
                    outlier_gene_col = 'Gene'
                    df_outlier = outliers_used.rename(columns={outlier_sample_col:'Sample', outlier_gene_col:'Gene'})
                    
                #----------- 调用函数auprc
                # print(df_outlier.head())
                
                if m == 'ABEILLE':
                    
                    df_pred = pd.read_csv(f"{row[m]}_raw.gz", sep='\t',index_col=0)
                    df_pred.index = [(s,g) for s,g in zip(df_pred['Sample'], df_pred['Transcript'])]
                    class_col = 'predict'
                    for score in abl_scores:
                        dict_auprc[k][f"{m}/{score}(class)"], dict_curve[k][f"{m}/{score}(class)"] = dict(), dict() 
                        dict_auprc[k][f"{m}/{score}(class)"][j], dict_curve[k][f"{m}/{score}(class)"][j] = sort_label_score_auprc(df_pred, class_col, score, df_outlier)
                        dict_auprc[k][f"{m}/{score}"], dict_curve[k][f"{m}/{score}"] = dict(), dict() 
                        dict_auprc[k][f"{m}/{score}"][j], dict_curve[k][f"{m}/{score}"][j] = sort_only_score_auprc(df_pred, score, df_outlier)
                
                if m == 'OUTRIDER':
                    
                    df_pred = pd.read_csv(f"{row[m]}_raw.gz", sep='\t',index_col=0)
                    df_pred.index = [(s,g) for s,g in zip(df_pred['sampleID'], df_pred['geneID'])]
                    class_col = 'aberrant'
                    for score in outrider_scores: # score = 'pValue'
                        dict_auprc[k][f"{m}/{score}"], dict_curve[k][f"{m}/{score}"] = dict(), dict() 
                        dict_auprc[k][f"{m}/{score}"][j], dict_curve[k][f"{m}/{score}"][j]  = sort_label_score_auprc(df_pred, class_col, score, df_outlier)
                
                if m == 'OUTSINGLE':
                    
                    df_pred = pd.read_csv(f"{row[m]}", sep='\t',index_col=0)
                    dict_auprc[k][m] ,dict_curve[k][m] = dict(), dict()
                    dict_auprc[k][m][j], dict_curve[k][m][j] = pred_outlier_auprc(df_pred, df_outlier)
                    
            else:
                pass
                # df_outlier = outliers_used.query('Sample in @df_pred.columns')
                
    file = f'{metric_output_path}/t{i:02d}_{t}_auprc.pkl'
    outputdata = [ t, tn, i, dict_auprc, dict_curve ]
    print(file)
    import pickle
    with open(file, 'wb') as f: 
        pickle.dump( outputdata,  f )
dict_auprc

t08_BRNAMY_s146_g14492 (196, 8)
ABEILLE
OUTRIDER
OUTSINGLE
/mnt/disk7t/xwj/axolotl_rev/result/dataset_gtex_nmd/metric/t08_BRNAMY_auprc.pkl
t18_BRNSNG_s132_g14884 (166, 8)
ABEILLE
OUTRIDER
OUTSINGLE
/mnt/disk7t/xwj/axolotl_rev/result/dataset_gtex_nmd/metric/t18_BRNSNG_auprc.pkl
t28_KDNCTX_s73_g15406 (138, 8)
ABEILLE
OUTRIDER
OUTSINGLE
/mnt/disk7t/xwj/axolotl_rev/result/dataset_gtex_nmd/metric/t28_KDNCTX_auprc.pkl
t46_UTERUS_s139_g14829 (244, 8)
ABEILLE
OUTRIDER
OUTSINGLE
/mnt/disk7t/xwj/axolotl_rev/result/dataset_gtex_nmd/metric/t46_UTERUS_auprc.pkl


{'all': {'ABEILLE/anomaly_score(class)': {0: 0.00011229500000000018},
  'ABEILLE/anomaly_score': {0: 0.00011147500000000013},
  'ABEILLE/delta_count(class)': {0: 0.00010041000000000008},
  'ABEILLE/delta_count': {0: 0.00010041000000000008},
  'ABEILLE/divergence_score(class)': {0: 0.00022090000000000052},
  'ABEILLE/divergence_score': {0: 0.00022090000000000052},
  'OUTRIDER/pValue': {0: 0.000814764999999999},
  'OUTRIDER/zScore': {0: 0.001829249999999996},
  'OUTRIDER/padjust': {0: 0.00039827500000000094},
  'OUTSINGLE': {0: 0.001670534999999994}}}

In [22]:
draw_methods = [
                'OUTRIDER/padjust', 
                'OUTSINGLE',
                
                'ABEILLE/anomaly_score', 
                'ABEILLE/delta_count', 
                'ABEILLE/divergence_score', 
                
                'ABEILLE/anomaly_score(class)', 
                'ABEILLE/delta_count(class)', 
                'ABEILLE/divergence_score(class)',
]

In [24]:
buffer =  []
for i, row in tissues.iterrows():
    t  = row.TISSUE_ID
    tn = row.TISSUE_NAME
    outlierfile = row.OUTLIER_FILE
    
    file = f'{metric_output_path}/t{i:02d}_{t}_auprc.pkl' # 改成需要的后缀 OUTRIDER_dim!
    
    with open(file,'rb') as f:
        data_load = pickle.load(f)
    t, tn, i, dict_auprc, dict_curve = data_load[0], data_load[1], data_load[2], data_load[3], data_load[4]
    k = 'all'
    for m in draw_methods:
        if m.startswith('OUTRIDER'):
            curve = dict_curve['all'][m][0].drop_duplicates().copy()
            aberrant = 'aberrant'
            mask = curve[aberrant] == True
            # print( curve.loc[mask,].tail(1) )
            buffer.append({
                'Method': m,
                'N_SAMPLE': row['N_SAMPLE'],
                'OUTLIER_FILE': os.path.basename(outlierfile),
                'N_Recall': curve[mask]['label'].sum(),
                'Precision':curve[mask]['precision'].values[-1],
                'Recall': curve[mask]['recall'].values[-1],
            })
            
        if m.startswith('OUTSINGLE'):
            print(m) 
            continue
            curve = dict_curve['all'][m][0].drop_duplicates().copy()
            print( curve.query('pred < 0.05').tail(1))
            # break
        if m.startswith('ABEILLE') & m.endswith('(class)'):
            curve = dict_curve['all'][m][0].drop_duplicates().copy()
            aberrant = 'predict'
            mask = curve[aberrant] == True
            # print( curve.loc[mask,].tail(1) )
            buffer.append({
                'Method': m,
                'N_SAMPLE': row['N_SAMPLE'],
                'OUTLIER_FILE': os.path.basename(outlierfile),
                'N_Recall': curve[mask]['label'].sum(),
                'Precision':curve[mask]['precision'].values[-1],
                'Recall': curve[mask]['recall'].values[-1],
            })
            # print( dict_curve['all'][m][0].query('@aberrant == True').tail(1) )
df_class_recall   = pd.DataFrame(buffer) 
df_class_recall

OUTSINGLE
OUTSINGLE
OUTSINGLE
OUTSINGLE


,Method,N_SAMPLE,OUTLIER_FILE,N_Recall,Precision,Recall
0,OUTRIDER/padjust,146,outlier_BRNAMY_sg196.tsv,3,0.0178,0.0306
1,ABEILLE/anomaly_score(class),146,outlier_BRNAMY_sg196.tsv,0,0.0000,0.0000
2,ABEILLE/delta_count(class),146,outlier_BRNAMY_sg196.tsv,0,0.0000,0.0000
3,ABEILLE/divergence_score(class),146,outlier_BRNAMY_sg196.tsv,0,0.0000,0.0000
4,OUTRIDER/padjust,132,outlier_BRNSNG_sg166.tsv,2,0.0300,0.0181
5,ABEILLE/anomaly_score(class),132,outlier_BRNSNG_sg166.tsv,0,0.0000,0.0000
6,ABEILLE/delta_count(class),132,outlier_BRNSNG_sg166.tsv,0,0.0000,0.0000
7,ABEILLE/divergence_score(class),132,outlier_BRNSNG_sg166.tsv,0,0.0000,0.0000
8,OUTRIDER/padjust,73,outlier_KDNCTX_sg138.tsv,1,0.0909,0.0072
9,ABEILLE/anomaly_score(class),73,outlier_KDNCTX_sg138.tsv,0,0.0000,0.0000
